In [ ]:
pip install 'pymongo[srv]'

In [ ]:
pip install python-dotenv

In [ ]:
pip install --upgrade google-cloud-datastore

In [ ]:
from google.cloud import datastore
import json  
from dotenv import load_dotenv
import os
from pymongo import TEXT, MongoClient

In [ ]:


if __name__ == "__main__":

    load_dotenv(override=True)

    # mongo client
    mongo_client = MongoClient(os.getenv('MONGO_URL'))

    db = mongo_client.sfeir_curriculum
    cv_collection = db.curriculums
    user_collection = db.users

    # datastore gcp client
    datastore_client = datastore.Client(namespace=os.getenv('DATASTORE_NAMESPACE'))

    # to query datastore db
    cvs_query = datastore_client.query(kind='curriculums')
    users_query = datastore_client.query(kind='users')

    # convert datastore entities to json
    curriculums = [ json.loads(json.dumps(cv)) for cv in list(cvs_query.fetch()) ]
    users = [ json.loads(json.dumps(user)) for user in list(users_query.fetch()) ]

    # save data in mongo
    cv_collection.insert_many(curriculums)
    user_collection.insert_many(users)

    # create mongo index in curriculums
    cv_collection.create_index([('$**', TEXT)], name='cv_index', default_language='french')


In [ ]:
input_search = 'full stack java angular'
load_dotenv(override=True)

# mongo client
mongo_client = MongoClient(os.getenv('MONGO_URL'))
db = mongo_client.sfeir_curriculum
cv_collection = db.curriculums
user_collection = db.users

result = list(cv_collection.find(
    {
        "$text": { "$search": input_search }
    },
    {
        "score": { "$meta": "textScore" }
    }
).sort( { "score": { "$meta": "textScore" } } ))

print(len(result))
result.reverse()
